目标：将全部的属性输入GBDT模型中，观察所有属性的重要性


目标的可行性存疑：
1、所有属性不可能被直接写入GBDT，必然存在很多的空值和无法数值化的值（字符串
2、会出现很多TRUE一类的需要进一步预处理的值（这个TRUE在jsonl文件中到底是怎么表示的（是宏定义的数字1还是什么？
目前的思路：
1、我自己在jsonl提取的时候，首先就删掉一些应该不需要的(比如所有的chat字符串等等)，这个可以减少csv的文件大小+减少进入模型去跑特征重要性的时候需要的数据预处理复杂度
2、进一步探讨……

In [2]:
import os #to access files
import pandas as pd #to work with dataframes
import numpy as np #just a tradition
from sklearn.model_selection import StratifiedKFold #for cross-validation
from sklearn.metrics import roc_auc_score #this is we are trying to increase
import matplotlib.pyplot as plt #we will plot something at the end
import seaborn as sns #same reason
import lightgbm as lgb #the model we gonna use

f:\Anaconda\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


从train_matches.jsonl中提取出所有的特征对应的所有数据项，成为一个csv文件

！！！！这个代码是错误的，不要跑，无法提取出想要的csv 提取的很混乱

In [1]:
'''
import json
import csv

# 定义输入 JSONL 文件和输出 CSV 文件的路径
PATH_TO_DATA = '../data/'

jsonl_file_path = os.path.join(PATH_TO_DATA,'input_test.jsonl')
csv_file_path = os.path.join(PATH_TO_DATA,'output_test.csv')

def flatten_json(nested_json, parent_key='', sep='-'):
    """递归地展开嵌套的 JSON 对象，并使用指定的分隔符."""
    items = []
    for k, v in nested_json.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_json(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            # 对列表内的每个元素展开
            for i, item in enumerate(v):
                if isinstance(item, dict):
                    items.extend(flatten_json(item, f"{new_key}{sep}{i}", sep=sep).items())
                else:
                    items.append((f"{new_key}{sep}{i}", item))
        else:
            items.append((new_key, v))
    return dict(items)

# 打开 JSONL 文件并读取数据
with open(jsonl_file_path, 'r', encoding='utf-8') as jsonl_file:
    # 读取 JSONL 文件中的每一行，并展开嵌套结构
    data = [flatten_json(json.loads(line)) for line in jsonl_file]

# 获取所有列名（字段）并写入 CSV 文件
fieldnames = sorted({key for row in data for key in row.keys()})
with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data)

print(f"数据已成功从 {jsonl_file_path} 转换为 {csv_file_path}，并展开了嵌套结构")
'''


NameError: name 'os' is not defined

导出了一个近乎10G的大csv文件，但是遇到一定的问题
1、文件过大，excel无法打开
2、在两行json转csv的时候可以保证正确性，2G的转化的时候正确性存疑

所以接下来写代码尝试只展示前三行csv

！！！！配合上面出错的代码的验证程序，也不要跑

In [2]:
'''
import pandas as pd

# 读取CSV文件
PATH_TO_DATA = '../data/'
csv_file_path = os.path.join(PATH_TO_DATA,'output_test.csv')

file_path = csv_file_path  # 请将 'your_file.csv' 替换为你的文件路径
#df = pd.read_csv(file_path)
df = pd.read_csv(csv_file_path, nrows=4)

# 展示表头和前三行数据
df.head(3)

# 将前三行写入一个新的CSV文件
output_file_path = os.path.join(PATH_TO_DATA, 'output_first_three_rows_demo.csv')
df.to_csv(output_file_path, index=False)

'''

"\nimport pandas as pd\n\n# 读取CSV文件\nPATH_TO_DATA = '../data/'\ncsv_file_path = os.path.join(PATH_TO_DATA,'output_test.csv')\n\nfile_path = csv_file_path  # 请将 'your_file.csv' 替换为你的文件路径\n#df = pd.read_csv(file_path)\ndf = pd.read_csv(csv_file_path, nrows=4)\n\n# 展示表头和前三行数据\ndf.head(3)\n\n# 将前三行写入一个新的CSV文件\noutput_file_path = os.path.join(PATH_TO_DATA, 'output_first_three_rows_demo.csv')\ndf.to_csv(output_file_path, index=False)\n\n"

提取数据 
将所有无折叠的一级标签提取出来  
对有折叠的一级标题，提取其内部包含了几组元素，即提取出其长度

In [4]:
import json
import csv

def extract_non_nested_and_count_nested_keys(json_object):
    """
    从JSON对象中提取所有不含嵌套属性的一级属性，
    并对包含嵌套属性的一级属性统计其内部的数据数量。
    """
    data = {}
    for key, value in json_object.items():
        # 如果属性不含嵌套（即不是dict或list），直接提取
        if not isinstance(value, (dict, list)):
            data[key] = value
        # 如果属性是列表，统计其长度并添加统计列
        elif isinstance(value, list):
            data[f"{key}_number"] = len(value)
        # 如果属性是字典类型，也记录其包含的键值数量
        elif isinstance(value, dict):
            data[f"{key}_number"] = len(value)
    return data

def jsonl_to_csv(jsonl_file_path, csv_file_path):
    """将JSONL文件转换为CSV文件，提取不含嵌套的一级属性，并统计嵌套属性的数量"""
    with open(jsonl_file_path, 'r') as jsonl_file, open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = None
        for line in jsonl_file:
            json_object = json.loads(line.strip())
            # 提取不含嵌套的一级属性和嵌套属性的数量
            processed_data = extract_non_nested_and_count_nested_keys(json_object)
            # 初始化CSV写入器（仅在第一次循环时执行）
            if csv_writer is None:
                csv_writer = csv.DictWriter(csv_file, fieldnames=processed_data.keys())
                csv_writer.writeheader()
            # 写入CSV行
            csv_writer.writerow(processed_data)

# 使用
PATH_TO_DATA = '../data/'
jsonl_file_path = os.path.join(PATH_TO_DATA,'input_test.jsonl')
csv_file_path = os.path.join(PATH_TO_DATA,'data_extract/main_table.csv')
jsonl_to_csv(jsonl_file_path, csv_file_path)


提取objectives表格

In [6]:
import json
import csv

def find_max_objectives(jsonl_file_path):
    """
    遍历JSONL文件，找出所有行中objectives的最大数量。
    """
    max_objectives = 0
    with open(jsonl_file_path, 'r') as jsonl_file:
        for line in jsonl_file:
            json_object = json.loads(line.strip())
            objectives_count = len(json_object.get("objectives", []))
            if objectives_count > max_objectives:
                max_objectives = objectives_count
    print(max_objectives)
    return max_objectives

def extract_objectives(json_object, max_objectives):
    """
    从JSON对象中提取所有objectives数据，将每个objective展开成单独的列。
    如果没有那么多objectives，则保留空值。
    """
    data = {}
    objectives = json_object.get("objectives", [])
    for i in range(1, max_objectives + 1):
        if i <= len(objectives):
            objective = objectives[i - 1]
            for key in ["time", "type", "player_slot", "team", "key", "slot"]:
                column_name = f"objective-{i}-{key}"
                data[column_name] = objective.get(key, "")
        else:
            for key in ["time", "type", "player_slot", "team", "key", "slot"]:
                column_name = f"objective-{i}-{key}"
                data[column_name] = ""
    return data

def jsonl_to_csv(jsonl_file_path, csv_file_path):
    """
    将JSONL文件转换为CSV文件，提取并展开每一行中的所有objective数据。
    """
    # 第一次遍历：找到最大的objectives数量
    max_objectives = find_max_objectives(jsonl_file_path)
    
    with open(jsonl_file_path, 'r') as jsonl_file, open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = None
        for line in jsonl_file:
            json_object = json.loads(line.strip())
            # 提取并展开objectives数据
            processed_data = extract_objectives(json_object, max_objectives=max_objectives)
            # 初始化CSV写入器（仅在第一次循环时执行）
            if csv_writer is None:
                csv_writer = csv.DictWriter(csv_file, fieldnames=processed_data.keys())
                csv_writer.writeheader()
            # 写入CSV行
            csv_writer.writerow(processed_data)

# 使用
PATH_TO_DATA = '../data/'
jsonl_file_path = os.path.join(PATH_TO_DATA,'input_test.jsonl')
csv_file_path = os.path.join(PATH_TO_DATA,'data_extract/objective_table.csv')
jsonl_to_csv(jsonl_file_path, csv_file_path)


27


同理，提取targets表格

In [10]:
import json
import csv
import os

def extract_targets(json_object):
    data = {}
    targets = json_object.get("targets", {})
    # 展开每个target属性
    for key in ["next_roshan_team", "radiant_win", "duration", "time_remaining", "game_time"]:
        data[key] = targets.get(key, "")
    return data

def jsonl_to_csv(jsonl_file_path, csv_file_path):
    """
    将JSONL文件转换为CSV文件，提取并展开每一行中的所有target数据。
     """
    with open(jsonl_file_path, 'r') as jsonl_file, open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = None
        for line in jsonl_file:
            json_object = json.loads(line.strip())
             # 提取并展开targets数据
            processed_data = extract_targets(json_object)
            # 初始化CSV写入器（仅在第一次循环时执行）
            if csv_writer is None:
                csv_writer = csv.DictWriter(csv_file, fieldnames=processed_data.keys())
                csv_writer.writeheader()
            csv_writer.writerow(processed_data)

# 使用
PATH_TO_DATA = '../data/'
jsonl_file_path = os.path.join(PATH_TO_DATA, 'input_test.jsonl')
csv_file_path = os.path.join(PATH_TO_DATA, 'data_extract/target_table.csv')
jsonl_to_csv(jsonl_file_path, csv_file_path)


同理，提取teamfights表格，其中，players涉及多级嵌套，将其嵌套展开单独成列

In [4]:
import json
import csv
import os

def extract_teamfights(json_object, max_players):
    """
    从JSON对象中提取teamfights数据，将每个teamfight和其下的player属性展开成单独的列。
    对于players中的多级字段（如ability_uses, item_uses等），保持为字典字符串。
    """
    data = {}
    teamfights = json_object.get("teamfights", [])

    for i, teamfight in enumerate(teamfights, start=1):
        # 提取teamfight的顶层字段
        data[f"teamfights-{i}-end"] = teamfight.get("end", "")
        data[f"teamfights-{i}-start"] = teamfight.get("start", "")
        data[f"teamfights-{i}-deaths"] = teamfight.get("deaths", "")
        data[f"teamfights-{i}-last_death"] = teamfight.get("last_death", "")

        # 提取每个player的数据
        players = teamfight.get("players", [])
        for j, player in enumerate(players, start=1):
            # 保持每个player的多级字段为字典字符串
            data[f"teamfights-{i}-player-{j}-ability_uses"] = json.dumps(player.get("ability_uses", {}))
            data[f"teamfights-{i}-player-{j}-deaths"] = player.get("deaths", "")
            data[f"teamfights-{i}-player-{j}-killed"] = json.dumps(player.get("killed", {}))
            data[f"teamfights-{i}-player-{j}-item_uses"] = json.dumps(player.get("item_uses", {}))
            data[f"teamfights-{i}-player-{j}-xp_delta"] = player.get("xp_delta", "")
            data[f"teamfights-{i}-player-{j}-damage"] = player.get("damage", "")
            data[f"teamfights-{i}-player-{j}-gold_delta"] = player.get("gold_delta", "")
            data[f"teamfights-{i}-player-{j}-deaths_pos"] = json.dumps(player.get("deaths_pos", {}))
            data[f"teamfights-{i}-player-{j}-healing"] = player.get("healing", "")
            data[f"teamfights-{i}-player-{j}-buybacks"] = player.get("buybacks", "")

    return data

def jsonl_to_csv(jsonl_file_path, csv_file_path):
    """
    将JSONL文件转换为CSV文件，提取并展开每一行中的所有teamfights数据。
    """
    all_fieldnames = set()
    max_players = 0
    
    # 第一次遍历：确定最大玩家数量和完整字段名
    with open(jsonl_file_path, 'r') as jsonl_file:
        for line in jsonl_file:
            json_object = json.loads(line.strip())
            teamfights = json_object.get("teamfights", [])
            max_players = max(max_players, max(len(tf.get("players", [])) for tf in teamfights) if teamfights else 0)
            processed_data = extract_teamfights(json_object, max_players)
            all_fieldnames.update(processed_data.keys())
    
    # 排序字段名，确保一致性
    all_fieldnames = sorted(all_fieldnames)
    
    # 第二次遍历：将数据写入CSV文件
    with open(jsonl_file_path, 'r') as jsonl_file, open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.DictWriter(csv_file, fieldnames=all_fieldnames)
        csv_writer.writeheader()
        for line in jsonl_file:
            json_object = json.loads(line.strip())
            processed_data = extract_teamfights(json_object, max_players)
            csv_writer.writerow(processed_data)

# 使用
PATH_TO_DATA = '../data/'
jsonl_file_path = os.path.join(PATH_TO_DATA, 'input_test.jsonl')
csv_file_path = os.path.join(PATH_TO_DATA, 'data_extract/teamfights_table.csv')
jsonl_to_csv(jsonl_file_path, csv_file_path)


提取players表(二级缩进版本）
即：如有缩进，提取到第二个缩进为止，其他暂时没有做进一步处理
我的思考：
1.涉及到time-key结构、名称-伤害-time结构的，是否可以变成数组？
2.xxx_log和xxx是重复的

In [10]:
import json
import csv
import os

def flatten_dict(data, prefix=""):
    """
    仅递归展开嵌套字典中需要展开的字段，避免过度拉平，保留指定字段为普通字段。
    如果字段是字典，则展开为`prefix-key`的形式，如果是其他类型，则直接保存为值。
    """
    flat_data = {}
    for key, value in data.items():
        new_key = f"{prefix}-{key}" if prefix else key
        if isinstance(value, dict) and key not in ["killed", "obs", "purchase", "actions"]:  # 不展开这些字段
            for sub_key, sub_value in value.items():
                flat_data[f"{new_key}-{sub_key}"] = sub_value
        else:
            flat_data[new_key] = value
    return flat_data

def extract_players(json_object):
    """
    从JSON对象中提取players数据，将每个player和其下的多级属性展开成单独的列。
    删除不需要的字段，保留需要展开的字段，并处理多级嵌套。
    """
    data = {}
    players = json_object.get("players", [])

    for i, player in enumerate(players, start=1):
        player_data = {}

        # 仅保留需要的字段
        required_fields = [
            "hero_inventory", "sen_placed", "hero_stash", "sen_left_log", "kills", "purchase_log",
            "obs_left_log", "max_hero_hit", "runes_log", "buyback_log", "obs_log", "max_mana",
             "creeps_stacked", "killed", "obs", "purchase", "y", "max_health",
            "sen_log", "hero_id", "kills_log", "towers_killed", "health", "xp_reasons", "randomed",
            "x", "gold_t", "rune_pickups", "level", "lh_t", "dn_t", "actions", "nearby_creep_death_count",
            "denies", "observers_placed", "times", "stuns", "deaths", "gold", "roshans_killed"
        ]
        
        for field in required_fields:
            if field in player:
                player_data[field] = player[field]

        # 展开需要的嵌套字典（但不展开指定字段）
        flat_player_data = flatten_dict(player_data, prefix=f"players-{i}")
        data.update(flat_player_data)

    return data

def jsonl_to_csv(jsonl_file_path, csv_file_path):
    """
    将JSONL文件转换为CSV文件，提取并展开每一行中的所有players数据。
    """
    all_fieldnames = set()
    
    # 第一次遍历：确定完整的字段名
    with open(jsonl_file_path, 'r') as jsonl_file:
        for line in jsonl_file:
            json_object = json.loads(line.strip())
            processed_data = extract_players(json_object)
            all_fieldnames.update(processed_data.keys())
    
    # 排序字段名，确保一致性
    all_fieldnames = sorted(all_fieldnames)
    
    # 第二次遍历：将数据写入CSV文件
    with open(jsonl_file_path, 'r') as jsonl_file, open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.DictWriter(csv_file, fieldnames=all_fieldnames)
        csv_writer.writeheader()
        for line in jsonl_file:
            json_object = json.loads(line.strip())
            processed_data = extract_players(json_object)
            csv_writer.writerow(processed_data)

# 使用
PATH_TO_DATA = '../data/'
jsonl_file_path = os.path.join(PATH_TO_DATA, 'input_test.jsonl')
csv_file_path = os.path.join(PATH_TO_DATA, 'data_extract/players_table_simpleversion.csv')
jsonl_to_csv(jsonl_file_path, csv_file_path)
